# Caller Intent Prediction

Within this notebook, we will predict the intent of a caller using historical data. This will allow Call Center Agents to be better prepared when faced with an incoming call.

We will be using the [Classification function from the suite of Snowflake's ML Functions](https://docs.snowflake.com/en/user-guide/ml-functions/classification) to build our model.
> - _Classification involves creating a classification model object, passing in a reference to the training data. The model is fitted to the provided training data. You then use the resulting schema-level classification model object to classify new data points and to understand the model’s accuracy through its evaluation APIs._
> - _The classification function is powered by a gradient boosting machine. For binary classification, the model is trained using an area-under-the-curve loss function. For multi-class classification, the model is trained using a logistic loss function._

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit", 
                     "name":"payer_call_center_assistant_v2", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

# Set session context 
session.use_role("SYSADMIN")

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

We will first create a view to store caller intent predictions.

In [ ]:
CREATE OR REPLACE VIEW CALLER_INTENT_CLASSIFICATION_VIEW AS
    SELECT 
        RECENT_ENROLLMENT_EVENT_IND,
        PCP_CHANGE_IND, 
        ACTIVE_CM_PROGRAM_IND,
        CHRONIC_CONDITION_IND,
        ACTIVE_GRIEVANCE_IND,
        ACTIVE_CLAIM_IND,
        POTENTIAL_CALLER_INTENT_CATEGORY
FROM CALLER_INTENT_TRAIN_DATASET;

Now, we will create an [ML Classification](https://docs.snowflake.com/en/user-guide/ml-functions/classification) function.

In [ ]:
CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION CALLER_INTENT(
    INPUT_DATA => SYSTEM$REFERENCE('view', 'CALLER_INTENT_CLASSIFICATION_VIEW'),
    TARGET_COLNAME => 'POTENTIAL_CALLER_INTENT_CATEGORY'
);

We will apply our ML model's prediction function on our prediction data.

In [ ]:
CREATE OR REPLACE TABLE CALLER_INTENT_PREDICTIONS AS
SELECT *, CALLER_INTENT!PREDICT(
    INPUT_DATA => {*})
    as predictions from CALLER_INTENT_PREDICT_DATASET;
SELECT * FROM CALLER_INTENT_PREDICTIONS;

We now create a view to join our member data with our predictions.

In [ ]:
CREATE OR REPLACE VIEW CALL_CENTER_MEMBER_DENORMALIZED_WITH_INTENT
AS
SELECT
    A.*,
    B.predictions:class::STRING AS POTENTIAL_CALLER_INTENT
FROM 
CALL_CENTER_MEMBER_DENORMALIZED A
LEFT OUTER JOIN 
CALLER_INTENT_PREDICTIONS B
ON A.MEMBER_ID = B.MEMBER_ID;

We will now cleanse some data for our sample members who will be part of the Streamlit demo.

In [ ]:
UPDATE CALL_RECORDINGS_TRANSCRIPT_TABLE 
SET TRANSCRIPT = replace(TRANSCRIPT, '159-568-6880','159568380')
WHERE TRANSCRIPT ILIKE '%Tracy%Smith%';;

In [ ]:
UPDATE CALL_CENTER_MEMBER_DENORMALIZED
SET CLAIM_PROVIDER = 'Howe Group'
WHERE GRIEVANCE_STATUS = 'Pending'
AND GRIEVANCE_TYPE = 'Inadequate Care'
AND CLAIM_PROVIDER in ('Thornton Group','Kent Group','Perez-Martinez');

### :tada: We now have `Caller Intent` predictions, which are ready to be used by the Streamlit App!